In [9]:
import pygame
import random
import time

# Load names from file
with open('names.txt', 'r') as f:
    names = f.read().split('\n')

# Pygame setup
pygame.init()
pixel_size = 142  # Change pixel size here
# Adjust screen dimensions so they are a multiple of the pixel size
screen_width = 9 * pixel_size
screen_height = 5 * pixel_size
screen = pygame.display.set_mode((screen_width, screen_height))

# Set up the font
font = pygame.font.Font(None, 18)
title_font = pygame.font.Font(None, 36)

# Define a grid of positions
positions = [(x, y) for x in range(0, screen_width, pixel_size) for y in range(0, screen_height, pixel_size)]
random.shuffle(positions)

# Create a dictionary mapping each name to a unique color and position
participants = {}
for i, name in enumerate(names):
    color = (random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))
    participants[name] = {'color': color, 'position': positions[i % len(positions)], 'displayed': False}

# Game loop
running = True
winner = None
displayed_participants = []  # List of displayed participants
start_time = time.time()
game_started = False
game_ended = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if not game_started:
        # Display countdown timer before the game starts
        current_time = time.time()
        elapsed_time = current_time - start_time
        remaining_time = 3 - int(elapsed_time)
        if remaining_time > 0:
            screen.fill((0, 0, 0))
            countdown_text = title_font.render(str(remaining_time), True, (255, 255, 255))
            countdown_rect = countdown_text.get_rect(center=(screen_width // 2, screen_height // 2))
            screen.blit(countdown_text, countdown_rect)
            pygame.display.flip()
            time.sleep(1)
        else:
            game_started = True
            start_time = time.time()  # Reset the start time for the game

    if game_started and not game_ended:
        # Display participants in the order they appear in the list
        if len(names) > 0:
            name = names.pop(0)
            pygame.draw.rect(screen, participants[name]['color'],
                             pygame.Rect(participants[name]['position'], (pixel_size, pixel_size)))
            text_surface = font.render(name, True, (0, 0, 0))
            text_rect = text_surface.get_rect(center=(participants[name]['position'][0] + pixel_size // 2,
                                                      participants[name]['position'][1] + pixel_size // 2))
            screen.blit(text_surface, text_rect)
            participants[name]['displayed'] = True
            displayed_participants.append(name)  # Add name to the displayed participants list
            pygame.display.flip()
            time.sleep(0.1 / 1.5)  # Make the game 1.5 times faster
        else:
            if winner is None and displayed_participants:
                winner = random.choice(displayed_participants)  # Choose a random winner from displayed participants
                game_ended = True

    if game_ended:
        # Display the name of the winner as a big title with a fun animation
        start_font_size = 36
        end_font_size = start_font_size * 2
        step_size = (end_font_size - start_font_size) / 30
        font_size = start_font_size
        for i in range(30):  # Animation length
            screen.fill((random.randint(50, 255), random.randint(50, 255), random.randint(50, 255)))
            title_font = pygame.font.Font(None, int(font_size))  # Increase font size dynamically
            winner_color = (random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))
            winner_text = title_font.render("Winner: " + winner, True, winner_color)
            winner_rect = winner_text.get_rect(center=(screen_width // 2, screen_height // 2))
            screen.blit(winner_text, winner_rect)
            pygame.display.flip()
            font_size += step_size
            time.sleep(0.1)
        time.sleep(2)  # Show the final frame for 2 seconds
        running = False

